In [1]:
from pynq import Overlay
import string
import pandas as pd
import numpy as np
from pynq import allocate

In [2]:
custom_hw = Overlay("newdmamatch.bit")
#custom_hw?

In [3]:
#set dma
dma_config = custom_hw.axi_dma_2
dma_datain = custom_hw.axi_dma_1
dma_dataout = custom_hw.axi_dma_0

In [4]:
def dma_check():
    dma_config?
    dma_datain?
    dma_dataout?

In [5]:
logn = 3
N = 2**logn

In [6]:
inbuffer = allocate(shape=(N,), dtype='uint32')
outbuffer = allocate(shape=(N,), dtype='uint32')
configbuffer = allocate(shape=(1,), dtype='uint32')

In [7]:
fftdata = np.random.randint(128, size=(N,), dtype=np.uint32)
#fftdata = np.array([128,128,128,128,128,128,128,128], dtype = 'uint32')
np.copyto(inbuffer, fftdata)

In [8]:
configbuffer

PynqBuffer([0], dtype=uint32)

In [9]:
def uint_to_cplx(buffer):
    cplx_ret = []
    only_real = []
    only_imag = []
    
    num_bits = 32
    complex_format_string = '({})+({}i)'
    convert_format_string = 'uint32:{}, complex form : {}, XN_RE_bit {}, XN_IM_bit {}, XN_RE_asbyte {}, XN_RE_fixed {}'
    num_bits = 32
    for x in buffer:
        x_32rawbit = f'{x:0{num_bits}b}'
        XN_IM_bits = x_32rawbit[0:16]
        XN_RE_bits = x_32rawbit[16:32]
        
        XN_RE_chk = int(XN_RE_bits, 2)
        
        
        XN_RE_bytes = int(XN_RE_bits, 2).to_bytes(2, byteorder = 'little')
        XN_IM_bytes = int(XN_IM_bits, 2).to_bytes(2, byteorder = 'little')
        
        XN_RE_fixed = int.from_bytes(XN_RE_bytes, byteorder = 'little', signed = True)
        XN_IM_fixed = int.from_bytes(XN_IM_bytes, byteorder = 'little', signed = True)
        
        XN_RE_fixed = XN_RE_fixed / 256.0
        XN_IM_fixed = XN_IM_fixed / 256.0
        
        cplx_form = complex_format_string.format(XN_RE_fixed, XN_IM_fixed)
        
        cplx_ret.append(cplx_form)
        only_real.append(XN_RE_fixed)
        only_imag.append(XN_IM_fixed)
        
        def check_by_print(chk):
            #internal check function
            if(chk == True):
                print(XN_RE_bits)
                print(XN_RE_chk)
                print(XN_RE_bytes)
                print(XN_RE_fixed)
                print(convert_format_string.format(
                    x, cplx_form, XN_RE_bits, XN_IM_bits, XN_RE_bytes, XN_RE_fixed
                    )
                )
        
        check_by_print(False)
        
    return cplx_ret,only_real,only_imag

In [10]:
test_input = np.array([32767,], dtype='uint32')
uint_to_cplx(test_input)

(['(127.99609375)+(0.0i)'], [127.99609375], [0.0])

In [11]:
inbuffer

PynqBuffer([ 7, 35,  3, 29, 31,  5,  0, 15], dtype=uint32)

In [12]:
inbuffer_cplx_form, inbuffer_only_real, inbuffer_only_imag = uint_to_cplx(inbuffer)
inbuffer_nparray = np.vectorize(complex)(inbuffer_only_real,inbuffer_only_imag)
inbuffer_cplx_form, inbuffer_nparray

(['(0.02734375)+(0.0i)',
  '(0.13671875)+(0.0i)',
  '(0.01171875)+(0.0i)',
  '(0.11328125)+(0.0i)',
  '(0.12109375)+(0.0i)',
  '(0.01953125)+(0.0i)',
  '(0.0)+(0.0i)',
  '(0.05859375)+(0.0i)'],
 array([0.02734375+0.j, 0.13671875+0.j, 0.01171875+0.j, 0.11328125+0.j,
        0.12109375+0.j, 0.01953125+0.j, 0.        +0.j, 0.05859375+0.j]))

In [13]:
configbuffer[0] = 0b000001110000000000000000 + logn

In [14]:
dma_config.sendchannel.transfer(configbuffer)

In [15]:
dma_config.sendchannel.wait()

In [16]:
%%timeit
dma_datain.sendchannel.transfer(inbuffer)
dma_dataout.recvchannel.transfer(outbuffer)
dma_datain.sendchannel.wait()
dma_dataout.recvchannel.wait()

705 µs ± 2.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [17]:
outbuffer

PynqBuffer([       125,      65493, 4294705187,     262179,    2293747,
            4293197789,    1900508, 4292804596], dtype=uint32)

In [18]:
_,output_only_real,output_only_imag = uint_to_cplx(outbuffer)
fft_hw_output = np.vectorize(complex)(output_only_real, output_only_imag)
fft_hw_output

array([ 0.48828125+0.j       , -0.16796875+0.j       ,
        0.13671875-0.015625j ,  0.13671875+0.015625j ,
       -0.05078125+0.1328125j, -0.13671875-0.109375j ,
       -0.140625  +0.109375j , -0.046875  -0.1328125j])

In [19]:
%%timeit
npresult = np.fft.fft(fftdata)

60 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
fft_sw_numpy = np.array(inbuffer_only_real)
fft_sw_output = np.fft.fft(fft_sw_numpy)
print(fft_sw_output)

[ 0.48828125+0.j         -0.04955583-0.13325273j  0.13671875+0.015625j
 -0.13794417-0.10981523j -0.16796875+0.j         -0.13794417+0.10981523j
  0.13671875-0.015625j   -0.04955583+0.13325273j]


In [21]:
def float32_to_fixed16(float32_val):
    float32_mul = float32_val*(256.0)
    float32_mul += 0.5
    float32_int = int(float32_mul)
    
    float32_asbytes = float32_int.to_bytes(2, byteorder = 'little',signed = True)
    float32_fixed = int.from_bytes(float32_asbytes, byteorder = 'little', signed = True)
    float32_fixed = float32_fixed / 256.0
    
    return float32_fixed
    

In [22]:
fft_real = fft_sw_output.real
fft_imag = fft_sw_output.imag

In [23]:
fft_real_fixed = []
fft_imag_fixed = []
for x in fft_real:
    fft_real_fixed.append(float32_to_fixed16(x))
for x in fft_imag:
    fft_imag_fixed.append(float32_to_fixed16(x))
#fft_real_fixed,fft_imag_fixed
fft_sw_fixed16 = np.vectorize(complex)(fft_real_fixed,fft_imag_fixed)
fft_sw_fixed16

array([ 0.48828125+0.j        , -0.046875  -0.12890625j,
        0.13671875+0.015625j  , -0.1328125 -0.10546875j,
       -0.1640625 +0.j        , -0.1328125 +0.109375j  ,
        0.13671875-0.01171875j, -0.046875  +0.1328125j ])

In [24]:
sorted_fft_hw_output = sorted(fft_hw_output)
sorted_fft_sw_output = sorted(fft_sw_output)
sorted_fft_sw_fixed16 = sorted(fft_sw_fixed16)

In [25]:
result_stacked = np.stack((sorted_fft_hw_output,sorted_fft_sw_output,sorted_fft_sw_fixed16), axis=1)
result_stacked

array([[-0.16796875+0.j        , -0.16796875+0.j        ,
        -0.1640625 +0.j        ],
       [-0.140625  +0.109375j  , -0.13794417-0.10981523j,
        -0.1328125 -0.10546875j],
       [-0.13671875-0.109375j  , -0.13794417+0.10981523j,
        -0.1328125 +0.109375j  ],
       [-0.05078125+0.1328125j , -0.04955583-0.13325273j,
        -0.046875  -0.12890625j],
       [-0.046875  -0.1328125j , -0.04955583+0.13325273j,
        -0.046875  +0.1328125j ],
       [ 0.13671875-0.015625j  ,  0.13671875-0.015625j  ,
         0.13671875-0.01171875j],
       [ 0.13671875+0.015625j  ,  0.13671875+0.015625j  ,
         0.13671875+0.015625j  ],
       [ 0.48828125+0.j        ,  0.48828125+0.j        ,
         0.48828125+0.j        ]])

In [26]:
ifft_sw_output = np.fft.ifft(fft_sw_output)
sorted(ifft_sw_output)

[0j,
 (0.01171875+0j),
 (0.019531249999999993+0j),
 (0.02734375+0j),
 (0.05859374999999999+0j),
 (0.11328125+0j),
 (0.12109375+0j),
 (0.13671875+0j)]

In [27]:
ifft_hw_output = np.fft.ifft(fft_hw_output)
sorted(ifft_hw_output)

[(0.02734375+0j),
 (0.04123050755684948+0.028079148757036078j),
 (0.04795987568698632-0.03435144908293905j),
 (0.05419921875-0.04443359375j),
 (0.05555574931301368-0.06818761341706095j),
 (0.05615234375+0.05419921875j),
 (0.0810546875+0.056640625j),
 (0.12478511744315052+0.008053663742963924j)]

In [28]:
ifft_fixed_output = np.fft.ifft(fft_sw_fixed16)
sorted(ifft_fixed_output)

[(0.00048828125-0.00048828125j),
 (0.01220703125-0.00048828125j),
 (0.020810680821793506+0j),
 (0.02978515625+0.00146484375j),
 (0.05803011282590298+0j),
 (0.11189176217409702+0j),
 (0.11962890625-0.00048828125j),
 (0.1354393191782065+0j)]

In [29]:
fft_mse_hw = np.square(np.subtract(sorted_fft_hw_output,sorted_fft_sw_fixed16)).mean()
fft_mse_sw = np.square(np.subtract(sorted_fft_sw_output,sorted_fft_sw_fixed16)).mean()
fft_mse_hw/fft_mse_sw

(-1524.8258354139052+3095.7364485210346j)

In [30]:
ifft_mse_hw = np.square(np.subtract(ifft_fixed_output,ifft_hw_output )).mean()
ifft_mse_sw = np.square(np.subtract(ifft_sw_output,ifft_fixed_output )).mean()
ifft_mse_hw/ifft_mse_sw

(892.8500628279717-684.8565795969612j)